In [ ]:
data  = np.load("/kaggle/input/de-lds-shuffled-seed/shuffled_data.npy")
labels_edited  = np.load("/kaggle/input/de-lds-shuffled-seed/shuffled_labels.npy")

max = np.max(data)
min = np.min(data)

#Normalizing data
data = data/max
data = (data - np.mean(data))/np.std(data)


#Splitting Dataset into train, validation, test 
train_labels = labels_edited[0:550]
test_labels = labels_edited[550:600]
final_labels = labels_edited[600:675]

train_data = data[0:550]
test_data = data[550:600]
final_test = data[600:675]



cf_labels = np.where(final_labels== -1, 2, final_labels) #Label in the form for confusion matrix
un, co = np.unique(cf_labels, return_counts=True)
print(f'Unique: {un}, Counts: {co}')

train_labels_reshaped = train_labels.reshape(-1,1) #Formatting for input to the CNN model
test_labels_reshaped = test_labels.reshape(-1,1)
final_labels_reshaped = final_labels.reshape(-1,1)

train_labels_reshaped = to_categorical(train_labels_reshaped, 3) #One Hot Encoding
test_labels_reshaped = to_categorical(test_labels_reshaped, 3)
final_labels_reshaped = to_categorical(final_labels_reshaped, 3)


import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import Sequential, clone_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.callbacks import ReduceLROnPlateau
import keras
import numpy as np
import time


model = Sequential([
    Conv2D(filters=64, kernel_size=5, input_shape=(62, 265, 5), padding='same'),
    Conv2D(filters=64, kernel_size=3, padding='same'),
    Conv2D(filters=64, kernel_size=3, padding='same'),
    MaxPooling2D(pool_size=2, strides=2),
    Dropout(rate=0.3),
    
    Conv2D(filters=128, kernel_size=3, padding='same'),
    Conv2D(filters=128, kernel_size=3, padding='same'),
    MaxPooling2D(pool_size=2, strides=2),
    Dropout(rate=0.2),
    
    Conv2D(filters=256, kernel_size=3, padding='same'),
    MaxPooling2D(pool_size=2, strides=2),
    Dropout(rate=0.25),
    
    Conv2D(filters=512, kernel_size=3, padding='same'),
    MaxPooling2D(pool_size=2, strides=2),
    Dropout(rate=0.3),
    
    Flatten(),
    
    Dense(512, activation='relu'),
    Dropout(rate=0.4),
    
    Dense(256, activation='relu'),
    Dropout(rate=0.2),
    
    Dense(64, activation='relu'),
    Dense(3, activation='softmax')
])
# Compile CNN Model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=4e-5),
              loss=keras.losses.categorical_crossentropy,
              metrics=['accuracy', tf.keras.metrics.RootMeanSquaredError()])
# Fit the CNN model
#reduce_lr_cnn = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=5, min_lr=1e-5)


history = model.fit(
    train_data,
    train_labels_reshaped,
    batch_size=64,
    epochs=100,
    validation_data=(test_data, test_labels_reshaped),
    #callbacks=[reduce_lr_cnn]
)


plt.figure(figsize=(12, 4))
dpi = 300

# Plot training & validation accuracy values
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()

#plt.savefig("Loss_Accuracy_curve_try2_1.png", dpi=dpi)
plt.show()